# Model Evaluation and Error Analysis

This notebook focuses on post-hoc evaluation and diagnostic analysis of the final melting-point prediction models, with particular emphasis on the stacked ensemble.

All models have already been trained and final test-set predictions exported prior to this analysis.  
The goal here is **not further optimisation**, but to understand:

- overall predictive performance on a strictly held-out test set
- how errors are distributed across the target range
- whether stacking improves robustness relative to individual base models
- where the model fails, and whether those failures are systematic or isolated
- which molecular features contribute most strongly to predictions

The analyses in this notebook are intended to support **model interpretability, reliability assessment, and scientific reasoning**, rather than further exhaustive optimization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mp.io import load_json

In [ ]:
from mp.io import get_repo_root

ROOT = get_repo_root()
fig_dir = ROOT / "reports" / "figures" / "Model_evaluation"

## Load evaluation artefacts

Load saved predictions, ensemble coefficients, and feature importances for post-hoc evaluation of the final models.  
All artefacts are reused from earlier steps; no models are trained in this notebook.

In [ ]:
final_test_df = pd.read_csv(ROOT/"data/processed/final_test.csv", index_col = 0)

stacker_weights = load_json(ROOT / "reports" / "stacker_weights.json")
weights = stacker_weights["weights"]

df_weights = (
    pd.DataFrame.from_dict(weights, orient="index", columns=["weight"])
    .rename_axis("model")
    .reset_index()
)

preds = pd.read_csv(ROOT / "reports" / "predictions" / "final_test_predictions.csv")

cat_fi = pd.read_csv(ROOT / "reports" / "feature_importance" / "catboost_feature_importance.csv")

### Parity plot: stacked ensemble

This parity plot compares predicted versus true melting points for the stacked ensemble on the final held-out test set.  
Points are coloured by absolute error to highlight the distribution of prediction errors across the target range.

The ensemble closely follows the ideal \(y = x\) relationship across most of the melting-point range, indicating good overall agreement and limited global bias.  

In [ ]:
abs_err = preds["abs_err_stack"].values

plt.figure(figsize=(8, 6))
sc = plt.scatter(
    y_true,
    y_pred,
    c=abs_err,
    s=20,
    alpha=0.7,
)

plt.plot(lims, lims, linestyle="--", linewidth=1)
plt.xlim(lims)
plt.ylim(lims)

plt.xlabel("True melting point (°C)")
plt.ylabel("Predicted melting point (°C)")
plt.title("Parity plot: stacked ensemble (final test set)")

cbar = plt.colorbar(sc)
cbar.set_label("Absolute error (°C)")

plt.tight_layout()
plt.savefig(fig_dir / "Parity_plot.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

### Residual distribution: stacked ensemble vs CatBoost

This figure compares the distribution of residuals for the stacked ensemble and the strongest individual base model (CatBoost) on the final test set.  
Residuals are centred around zero for both models, indicating limited systematic bias.

The stacked ensemble shows a slightly narrower and more concentrated residual distribution, with reduced mass in the tails relative to CatBoost.  
This indicates variance reduction through stacking.

In [ ]:
r_stack = preds["resid_stack"].to_numpy()
r_cat = preds["resid_cat"].to_numpy()

# Optional: clip extreme tails so the histogram isn't dominated by a few outliers
# (keeps the plot interpretable; remove if you prefer raw)
clip_pct = 99.5
lim = np.percentile(np.abs(np.concatenate([r_stack, r_cat])), clip_pct)
r_stack_plot = np.clip(r_stack, -lim, lim)
r_cat_plot = np.clip(r_cat, -lim, lim)

plt.figure(figsize=(8, 5))

bins = 40
plt.hist(r_cat_plot, bins=bins, alpha=0.5, density=True, label="CatBoost residuals")
plt.hist(r_stack_plot, bins=bins, alpha=0.5, density=True, label="Stacked ensemble residuals")

plt.axvline(0, linestyle="--", linewidth=1)

plt.xlabel("Residual (predicted − true) (°C)")
plt.ylabel("Density")
plt.title("Residual distribution: stacked ensemble vs CatBoost (final test set)")
plt.legend()

plt.tight_layout()
plt.savefig(fig_dir / "Residual_distribution.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

### Absolute error distribution across models

This violin plot compares the distribution of absolute prediction errors across all base models and the stacked ensemble on the final test set.  
The stacked ensemble exhibits a lower median absolute error and a more concentrated error distribution relative to the individual models.

In particular, stacking reduces the frequency of large errors, indicating improved robustness rather than isolated gains on a small subset of compounds.  
This supports the use of an ensemble approach for melting-point prediction, where individual models capture complementary aspects of the data.

In [ ]:
model_labels = ["CatBoost", "XGBoost", "kNN", "Neural network", "Stacked"]

plt.figure(figsize=(8, 5))
plt.violinplot(
    data_plot,
    showmeans=False,
    showmedians=True,
    showextrema=False,
)

plt.xticks(
    ticks=range(1, len(model_labels) + 1),
    labels=model_labels,
)

plt.ylabel("Absolute error (°C)")
plt.title("Absolute error distribution across models (final test set)")
plt.tight_layout()
plt.savefig(fig_dir / "Absolute_error_distribution.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

### Residuals vs predicted melting point (stacked ensemble)

This plot shows residuals as a function of the predicted melting point for the stacked ensemble, allowing assessment of systematic trends across the prediction range.

Residuals are centred around zero throughout, indicating no strong prediction bias as a function of melting point.  
A modest increase in spread is observed at higher predicted values, suggesting increased uncertainty in regions of sparser chemical coverage rather than a distinct failure regime.

In [ ]:
x = preds["pred_stack"].to_numpy()
y = preds["resid_stack"].to_numpy()

# Optional: clip extreme residuals for readability
clip_pct = 99.5
lim = np.percentile(np.abs(y), clip_pct)
y_plot = np.clip(y, -lim, lim)

plt.figure(figsize=(7, 5))
plt.scatter(
    x,
    y_plot,
    s=15,
    alpha=0.5,
)



plt.axhline(0, linestyle="--", linewidth=1)

plt.xlabel("Predicted melting point (°C)")
plt.ylabel("Residual (predicted − true) (°C)")
plt.title("Residuals vs predicted melting point (stacked ensemble)")

plt.tight_layout()
plt.savefig(fig_dir / "Residuals_vs_predicted_melting_point.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

### Worst-case absolute errors (stacked ensemble)

This table lists the top 15 test-set compounds with the largest absolute prediction errors for the stacked ensemble.  
The purpose of this inspection is to assess whether the largest errors reflect systematic failure modes or isolated, compound-specific deviations.

The worst errors are distributed across a wide range of true melting points and include both over- and under-predictions.  
This suggests that large deviations are not driven by a single target-range effect, but are more likely associated with specific chemical or solid-state features not fully captured by the available descriptors.

In [ ]:
N = 15 

cols = ["y_true", "pred_stack", "abs_err_stack", "resid_stack"]
worst = (
    preds[cols]
    .sort_values("abs_err_stack", ascending=False)
    .head(N)
    .copy()
)

# Round for readability
worst[["y_true", "pred_stack", "abs_err_stack", "resid_stack"]] = worst[
    ["y_true", "pred_stack", "abs_err_stack", "resid_stack"]
].round(2)

worst

### Stacker weights (model contributions)

This plot shows the coefficients learned by the ridge regression meta-learner when combining predictions from the base models.  
Coefficients reflect the relative contribution of each model to the stacked ensemble after standardisation of the meta-features.

Multiple base models receive non-zero weight, indicating that the ensemble leverages complementary predictive signals rather than relying on a single dominant model.  
These coefficients should be interpreted qualitatively, as they depend on feature scaling and regularisation, and do not represent absolute model importance.

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(df_weights["model"], df_weights["weight"])

plt.axhline(0, linestyle="--", linewidth=1)
plt.ylabel("Stacker coefficient")
plt.xlabel("Base model")
plt.title("Stacker weights (ridge regression meta-learner)")

plt.tight_layout()
plt.savefig(fig_dir / "stacker_weights.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

### CatBoost feature importance

This figure shows the top 20 features ranked by importance in the CatBoost model, providing insight into which molecular descriptors most strongly influence melting-point predictions.

Highly ranked features relate to molecular polarity (e.g. TPSA, hydrogen-bonding descriptors), molecular size and shape (e.g. molecular weight, Bertz complexity), and ring structure.  
These factors are consistent with physical intuition for melting point, reflecting the role of intermolecular interactions and solid-state packing rather than specific functional groups alone.

In [ ]:
N = 20  # top-N features to show

fi_top = cat_fi.sort_values("importance", ascending=False).head(N)

plt.figure(figsize=(7, 5))
plt.barh(
    fi_top["feature"][::-1],       
    fi_top["importance"][::-1],
)

plt.xlabel("Feature importance (CatBoost)")
plt.title(f"Top {N} CatBoost feature importances")

plt.tight_layout()
plt.savefig(fig_dir / "CatBoost_feature_importance.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

## Summary and conclusions

This notebook presented a post-hoc evaluation of the final melting-point prediction models, with emphasis on the stacked ensemble.

Across multiple complementary diagnostics, the ensemble demonstrates improved robustness relative to individual base models, with reduced error variance and limited systematic bias on the held-out test set.  
Inspection of residuals and worst-case errors suggests that remaining failures are sparse and likely driven by compound-specific chemical or solid-state effects rather than target-range extrapolation.

Overall, these results support the use of ensemble modelling for melting-point prediction and provide a transparent assessment of model reliability and limitations.